In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
import torch.nn.functional as F

In [4]:
F.softmax

<function torch.nn.functional.softmax(input: torch.Tensor, dim: Optional[int] = None, _stacklevel: int = 3, dtype: Optional[int] = None) -> torch.Tensor>

In [5]:
import attr

In [10]:
@attr.s
class C(object):
    d = {2: "hej"}
    x: F.softmax = attr.ib(init=False, default=F.softmax)
    @x.validator
    def check(self, attribute, value):
        print(attribute)
        print(self.x)

In [14]:
c = C()

Attribute(name='x', default=<function softmax at 0x7fb624839ca0>, validator=<function C.check at 0x7fb622ce2040>, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=False, metadata=mappingproxy({}), type=<function softmax at 0x7fb624839ca0>, converter=None, kw_only=False, inherited=False, on_setattr=None)
<function softmax at 0x7fb624839ca0>


In [12]:
c.x

<function torch.nn.functional.softmax(input: torch.Tensor, dim: Optional[int] = None, _stacklevel: int = 3, dtype: Optional[int] = None) -> torch.Tensor>

In [26]:
from torch import nn

In [33]:
l = nn.ModuleList([])

In [34]:
f = nn.Linear(10, 10)

In [35]:
for _ in range(10):
    l.append(f)

In [36]:

l

ModuleList(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): Linear(in_features=10, out_features=10, bias=True)
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): Linear(in_features=10, out_features=10, bias=True)
  (8): Linear(in_features=10, out_features=10, bias=True)
  (9): Linear(in_features=10, out_features=10, bias=True)
)

In [37]:
ff = f.copy()

AttributeError: 'Linear' object has no attribute 'copy'

In [38]:
from copy import deepcopy

In [39]:
ff = deepcopy(f)

In [43]:
id(ff)

140011688939472

In [42]:
id(f)

140011688936544

In [44]:
id(l[0])

140011688936544

In [45]:
id(l[1])

140011688936544

In [58]:
".".join("nn.LayerNorm".split(".")[:-1])

'nn'

In [60]:
getattr("torch.nn", "LayerNorm")

AttributeError: 'str' object has no attribute 'LayerNorm'